https://medium.com/analytics-vidhya/simplifying-social-media-sentiment-analysis-using-vader-in-python-f9e6ec6fc52f
https://towardsdatascience.com/almost-real-time-twitter-sentiment-analysis-with-tweep-vader-f88ed5b93b1c
https://python.plainenglish.io/twitter-sentiment-analysis-using-vader-tweepy-b2a62fba151e
https://www.analyticsvidhya.com/blog/2021/06/twitter-sentiment-analysis-a-nlp-use-case-for-beginners/

In [32]:
# !pip install vaderSentiment
# !pip install twitter

     |████████████████████████████████| 50 kB 1.7 MB/s eta 0:00:01


In [44]:
import requests
import base64
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from twitter import *

In [116]:
api_vars = get_dataframe("twitter.pkl")

CLIENT_KEY = api_vars.client_key[0]
CLIENT_SECRET =  api_vars.client_secret[0]
ACCESS_TOKEN = api_vars.access_token[0]
ACCESS_TOKEN_SECRET = api_vars.token_secret[0]

TOPIC = "Lockheed -filter:retweet"

MAX_TWEETS = 400
TWEET_LIMIT = 100 # we can get only 100 tweet per call with standard search api

PICKLE_FILENAME = "lockheed_tweets.pkl"

In [94]:
def tweet_generator(tweets):
    for tweet in tweets:
        modified_tweet= dict(tweet)

        modified_tweet['user_id'] = tweet['user']['id']
        del modified_tweet['user']
        
        if 'retweeted_status' in tweet:
            modified_tweet['retweeted_status_id'] = tweet['retweeted_status']['id']
            del modified_tweet['retweeted_status']
        
        yield modified_tweet

def save_to_file(ts, file_name):
    ts.to_pickle(file_name)
    return None

def get_dataframe(file_name):
    return pd.read_pickle(file_name)

In [81]:
key_secret = '{}:{}'.format(CLIENT_KEY, CLIENT_SECRET).encode('ascii')
b64_encoded_key = base64.b64encode(key_secret)
b64_encoded_key = b64_encoded_key.decode('ascii')

base_url = 'https://api.twitter.com/'
auth_endpoint = base_url+'oauth2/token'

auth_headers = { 'Authorization': 'Basic {}'.format(b64_encoded_key),
                'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}

auth_data = { 'grant_type': 'client_credentials'}

response = requests.post(auth_endpoint, headers=auth_headers, data=auth_data)
json_data =  response.json()
access_token = json_data['access_token']

search_headers = {'Authorization': 'Bearer {}'.format(access_token)}

In [117]:
try:
    print("Reading from file.")
    tweet_df = get_dataframe(PICKLE_FILENAME)
    print(f"File read. Retrieved {len(tweet_df)} tweets.")
except:
    print("Error reading from file.")
    parameters = { 'q': TOPIC, 'result_type': 'all', 'count': TWEET_LIMIT }
    search_url = base_url+'1.1/search/tweets.json'
    response = requests.get(search_url, headers=search_headers, params=parameters)
    print("Getting tweets.")
    tweet_response = response.json()
    ids = [tw['id'] for tw in tweet_response['statuses']]
    tw_ids = []
    tw_ids += ids

    temp = []

    temp += tweet_response['statuses']
    print('Total tweets retrieved= {}'.format(len(temp)))
    
    tweet_df = pd.DataFrame(tweet_generator(temp))

min_id = tweet_df.tail(1).id.values[0]
print(f"Min ID: {min_id}.")

Reading from file.
File read. Retrieved 400 tweets.
Min ID: 1451566522439135238.


In [118]:
temp = []

for  i in range(MAX_TWEETS//TWEET_LIMIT -1):
    parameters = { 'q': TOPIC, 'result_type': 'recent', 'count': TWEET_LIMIT, 'max_id': min_id}
    print("searching tweets with id  < {}".format(min_id))
    search_url = base_url+'1.1/search/tweets.json'
    try:
        print("Getting tweets.")
        response = requests.get(search_url, headers=search_headers, params=parameters)
        tweet_response = response.json()
        print("Got tweets.")
    except:
        print("Something bad happened.")
        break
    
    ids = [tw['id'] for tw in tweet_response['statuses']]
    tw_ids += ids
    print(f"Round #{i}; Number of tweets: {len(tweet_response['statuses'])}")
    if min_id == min(ids):
        print("We are not progressing.")
        break
    min_id = min(ids)
    temp += tweet_response['statuses']

print("All Done.")
tweet_df = tweet_df.append(pd.DataFrame(tweet_generator(temp)), ignore_index=True)
#save_to_file(tweet_df, PICKLE_FILENAME)

searching tweets with id  < 1451566522439135238
Getting tweets.
Got tweets.
Round #0; Number of tweets: 100
searching tweets with id  < 1451546943704940554
Getting tweets.
Got tweets.
Round #1; Number of tweets: 100
searching tweets with id  < 1451526291342217217
Getting tweets.
Got tweets.
Round #2; Number of tweets: 96
All Done.


In [119]:
print('Total tweets retrieved= {}'.format(len(tweet_df)))
save_to_file(tweet_df, PICKLE_FILENAME)

Total tweets retrieved= 696


In [120]:
tweet_df = get_dataframe(PICKLE_FILENAME)
len(tweet_df)

696

In [136]:
tweet_df.head(3)

,created_at,id,id_str,text,truncated,entities,metadata,source,in_reply_to_status_id,in_reply_to_status_id_str,...,favorited,retweeted,lang,user_id,retweeted_status_id,possibly_sensitive,extended_entities,quoted_status_id,quoted_status_id_str,quoted_status
0,Fri Oct 22 22:20:59 +0000 2021,1451674974280949761,1451674974280949761,RT @gustinicchi: Lavoratori lasciano in blocco...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'it', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,...,False,False,it,1397217753317224450,1.451617e+18,NaN,NaN,NaN,NaN,NaN
1,Fri Oct 22 22:20:10 +0000 2021,1451674770676801540,1451674770676801540,"‘LASER DOME’: Rafael, Lockheed Developing Next...",True,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,...,False,False,en,873959210274443264,NaN,False,NaN,NaN,NaN,NaN
2,Fri Oct 22 22:11:58 +0000 2021,1451672707221819392,1451672707221819392,#PlaneAlert ICAO: #AE03D5 Tail: #164598 \nOwne...,True,"{'hashtags': [{'text': 'PlaneAlert', 'indices'...","{'iso_language_code': 'fr', 'result_type': 're...","<a href=""https://ramonk.net/planefence"" rel=""n...",NaN,None,...,False,False,fr,1259995501916819459,NaN,False,NaN,NaN,NaN,NaN


In [134]:
analyser = SentimentIntensityAnalyzer()

for index, tweet in tweet_df.head(100).iterrows():
    score = analyser.polarity_scores(tweet.text)
    print(f"{str(score)}")

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 0.897, 'pos': 0.103, 'compound': 0.128}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.087, 'neu': 0.913, 'pos': 0.0, 'compound': -0.25}
{'neg': 0.087, 'neu': 0.913, 'pos': 0.0, 'compound': -0.25}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 0.867, 'pos': 0.133, 'compound': 0.25}
{'neg': 0.0, 'neu': 0.92, 'pos': 0.08, 'compound': 0.0772}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
